# 로이터 뉴스 데이터셋

케라스의 [tf.keras.datasets.reuters](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/reuters)로부터 로이터 뉴스 데이터셋을 불러오겠습니다.  
역시 등장빈도가 10,000등안에 들지 않는 단어들은 `<UNKNOWN>`으로 전처리하겠습니다.
- 로이터 뉴스 기사 11,228개
- 훈련용 : 8,982개, 테스트용 : 2,246개
- 46개의 주제로 분류되며 라벨은 0부터 45
- 주제별 기사 개수는 편차가 큼  
![](https://drive.google.com/thumbnail?id=1ELjNVEQLqoPw7g6TXLINYz0seROlrJd0&sz=s4000)

In [ ]:
from tensorflow import keras
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print(f"훈련용 : {len(train_data)}")
print(f"테스트용 : {len(test_data)}")

0부터 45까지의 라벨은 다음 주제와 대응합니다.

In [ ]:
 class_names = ['cocoa','grain','veg-oil','earn','acq','wheat','copper','housing','money-supply',
                'coffee','sugar','trade','reserves','ship','cotton','carcass','crude','nat-gas',
                'cpi','money-fx','interest','gnp','meal-feed','alum','oilseed','gold','tin',
                'strategic-metal','livestock','retail','ipi','iron-steel','rubber','heat','jobs',
                'lei','bop','zinc','orange','pet-chem','dlr','gas','silver','wpi','hog','lead']

주제별 기사 개수는 편차가 매우 큽니다.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,8))
plt.hist(train_labels, bins=46, align='left')
plt.xticks(range(46))
plt.show()

**[실습1] (10분) key를 class_names의 항목으로 value를 해당 기사 개수로 가지는 딕셔너리를 출력하시오.**

첫번째 훈련 데이터입니다.  
원래는 영문 기사인데 이미 전처리되어 정수열로 저장되어 있습니다.

In [ ]:
print(train_data[0])

무슨 말인지는 모르겠지만 주제가 earn이라네요.

In [ ]:
print(class_names[train_labels[0]])

영어를 인덱스로 바꾸는 사전을 불러오겠습니다.  
얼마나 자주 등장하느냐로 어휘에 인덱스를 부여합니다.  
인덱스는 1부터 출발합니다.

In [ ]:
word_index = reuters.get_word_index()
print(word_index)

위에서 키와 밸류를 바꾼 딕셔너리를 만들겠습니다.  
인덱스를 영어로 바꾸는데 필요한 사전입니다.

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index)

등장빈도 톱 20위 어휘들입니다.  
일상적으로 많이 쓰이는 관사, 전치사, 대명사가 보이네요.  
아무래도 경제 기사들이다보니 dlrs(=dollars), pct(=percent), mln(=million), 숫자들이 등장합니다.  
인터뷰와 인용 때문인지 said가 상위권입니다.

In [ ]:
for idx in range(1,21):
    print(reverse_word_index[idx])

위 사전을 이용해서 첫번째 훈련 데이터를 디코딩해보죠.  
주의할 점은 전처리된 데이터에서 0,1,2,3은 특별 토큰을 나타냅니다.
- 0 : `<PAD>` (문장의 길이가 같아지도록 끼워넣는 더미, 여기서는 사용안함)
- 1 : `<START>` (문장의 시작을 알림)
- 2 : `<UNK>` (등장빈도가 낮은 어휘)
- 3 : `<UNUSED>`

그래서 어휘들의 인덱스가 3씩 뒤로 밀려있습니다.  
예를들어, the의 인덱스는 원래 1인데 특별 토큰때문에 3만큼 밀려서 4로 표시됩니다.  
그래서, 3을 빼준후에 어휘로 바꿔줘야 제대로 디코딩이 됩니다.  
`.get(i-1,"?")`은 i-1이 딕셔너리의 key에 없으면 ?로 대체하라는 뜻입니다.  
i가 특수토큰을 나타내는 0,1,2,3일경우는 i-3이 -3,-2,-1,0이 되어서 딕셔너리의 key에 없기 때문에 ?로 대체됩니다.  
" ".join은 리스트 안의 단어를을 이어붙이는데 사이에 따옴표안에 있는 문자인 스페이스를 끼워 넣으라는 뜻입니다.

In [ ]:
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])
print(decoded_newswire)

기사마다 데이터의 길이가 다릅니다.  
이래서는 신경망의 입력 뉴런의 수가 정해질 수 없습니다.

In [ ]:
import numpy as np

news_length = [len(news) for news in train_data]

print(f'최대 길이 : {np.max(news_length)}')
print(f'평균 길이 : {np.mean(news_length)}')
print(f'최소 길이 : {np.min(news_length)}')

plt.hist(news_length, bins=50)
plt.show()

모두 10,000의 길이를 가지는 멀티 핫 벡터로 다시 인코딩하겠습니다.  
정수형 라벨을 그대로 사용해도 되지만 연습삼아 원 핫 인코딩한 후에 학습시키겠습니다.

In [ ]:
from tensorflow.keras.utils import to_categorical

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

멀티 핫 인코딩한 첫번째 훈련 데이터입니다.

In [ ]:
np.set_printoptions(linewidth=100,threshold=10000)

print(x_train[0])

# 다중분류

다음과 같이 신경망을 구성하겠습니다.  
![](https://drive.google.com/thumbnail?id=1r-y981q0FqfJRTUR0rgymLFpddrUnJLf&sz=s4000)

In [ ]:
from keras import models
from keras.layers import Dense
from tensorflow.keras.utils import plot_model

model = keras.Sequential([
    Dense(64, input_shape=(10000,), activation="relu"),
    Dense(64, activation="relu"),
    Dense(46, activation="softmax")
])

plot_model(model, show_shapes=True, show_layer_activations=True)

라벨을 원 핫 인코딩했기 때문에 손실함수 설정을 categorical_crossentropy로 해줍니다.

In [ ]:
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

8,982개의 훈련 데이터를 1,000개와 7,982개로 쪼개서 전자를 검증용으로 후자를 훈련용으로 사용하겠습니다.

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

검증 데이터로 하이퍼 파라미터인 학습회수를 결정하겠습니다.  
학습회수를 넉넉히 20에폭
으로 잡고 과적합을 유도하겠습니다.

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

대략 9에폭부터 과적합이 시작되네요.

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

신경망을 초기화하고 훈련 에폭을 9로 설정합시다.  
하이퍼 파라미터 튜닝이 끝났기 때문에 처음에 주어진 훈련 데이터 전체를 사용해 학습시키겠습니다.  
46진 분류라는 점을 감안하면 최종 정확도가 매우 높게 느껴질수 있습니다.  
주제별 기사 개수가 고르다면 이는 분명 높은 정확도가 맞습니다.  
기사를 안읽고 찍었을때 대략 1/46$\approx$0.021가 나올테니까요.  
하지만 위에서 봤다시피 주제별 기사개수의 편차가 매우 큽니다.

In [ ]:
model = keras.Sequential([
  Dense(64, input_shape=(10000,), activation="relu"),
  Dense(64, activation="relu"),
  Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.fit(x_train,
          y_train,
          epochs=9,
          batch_size=512)

results = model.evaluate(x_test, y_test)
print("\n")
print(results)

기사 내용을 안읽고 주제를 찍는 실험을 해보겠습니다.  
주의할 점은 찍는 주제를 기사 개수만큼 답해야 합니다.  
예를 들어, cocoa는 55번 답하고 grain은 432번 답합니다.  
구현하는 아이디어는 라벨을 섞은후 원래 라벨과 순서대로 비교해서 일치하는 개수를 세는 것입니다.  
섞은 후의 리스트에 따라 순서대로 대답한다고 생각하는 것이지요.  
클래스 편차로 인해 이런 방식으로 찍을 경우 예상외로 높은 정확도가 나옵니다.  
실행할때미다 조금씩 달라지지만 무려 MNIST 경우의 두배에 가깝네요.

In [ ]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array.mean()

실행할때마다 달라지긴 하는데 첫번째 테스트 데이터에 대해서는 신경망이 3(=earn) 또는 4(=acq)중에 망설입니다.

In [ ]:
predictions = model.predict(x_test)

plt.bar(range(46), predictions[0])
plt.show()
print(f"예측 주제 : {class_names[np.argmax(predictions[0])]}")

라벨은 3(=earn)입니다.  
학습을 새로 할때마다 맞출때도 있고 틀릴때도 있습니다.  
틀릴 때는 4(=acq)라고 대답해서입니다.

In [ ]:
print(f"실제 주제 : {class_names[test_labels[0]]}")
print("\n텍스트 :")
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in test_data[0]])
print(decoded_newswire)

우연이 아니라 혼동행렬을 구해보면 3를 4이라고 대답한 경우가 많습니다.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

confusion = confusion_matrix(test_labels, np.argmax(predictions, axis=1))

plt.figure(figsize=(20,20))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, np.argmax(predictions, axis=1), zero_division=0))

**[실습2] (10분) (i) 등장빈도가 100등안에 들지 못하는 단어는 `<UNK>`으로 전처리한 후 첫번째 훈련 데이터를 디코딩하여 출력하시오.**

**(ii) 멀티 핫 인코딩한 후 첫번째 훈련 데이터를 출력하시오.**

**(iii) 입력 뉴런수만 바꾼 신경망으로 학습시킨후 테스트 데이터의 정확도를 측정하시오.**

# 병목(bottleneck)현상

두번째 Affine층의 출력 뉴런수를 64개에서 4개로 줄여보겠습니다.  
이는 마치 고속도로 차선이 갑자기 줄어드는 것과 같습니다.  
교통체증이 발생하겠죠.  
신경망도 마찬가지입니다.  
병목(bottleneck)현상때문에 정확도가 많이 떨어집니다.  
![](https://drive.google.com/thumbnail?id=1Qqu_Qk3-DNMvT7r8okFdiYSRx0agLnVC&sz=s4000)

In [ ]:
model_bottle = keras.Sequential([
    Dense(64, activation="relu"),
    Dense(4, activation="relu"),
    Dense(46, activation="softmax")
])

model_bottle.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model_bottle.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))

results = model_bottle.evaluate(x_test, y_test)
print("\n")
print(results)